<a href="https://colab.research.google.com/github/adalves-ufabc/2025.Q3-PLN/blob/main/2025_Q3_PLN_AULA_15_Notebook_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

## **Busca Semântica [LangChain]**
---



In [ ]:
#@title Instalando o pacote LangChain
!pip install langchain -q U

In [ ]:
#@title Versão do LangChain

import langchain

print(langchain.__version__)

0.3.27


In [ ]:
#@title Integração com o pacote da OpenAI

!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.9/469.9 kB 14.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.3 which is incompatible.


In [ ]:
#@title Definindo a chave da API da OpenAI

import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
#@title Definindo a chave da API da OpenAI
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


## **Busca Semântica**

Uma busca semântica de texto é um tipo de pesquisa que entende o significado das palavras e frases — não apenas as palavras exatas digitadas.

👉 Em outras palavras, em vez de procurar apenas correspondências literais de texto (como faz uma busca tradicional por palavras-chave), a busca semântica tenta interpretar a intenção do usuário e o contexto por trás da consulta.

🧠 Como funciona

A busca semântica normalmente usa técnicas de Processamento de Linguagem Natural e modelos de aprendizado de máquina (como *`embeddings`* de texto) para representar frases como vetores numéricos em um espaço de significados.

Assim, textos com significados semelhantes ficam “perto” uns dos outros nesse espaço — mesmo que usem palavras diferentes.

Este exemplo irá familiarizá-lo com as abstrações do **LangChain** de *`document loader`*, *`embedding`* e *`vector store`*. Essas abstrações são projetadas para suportar a recuperação de dados — de bancos de dados (vetoriais) e outras fontes — para integração com fluxos de trabalho LLM. Elas são importantes para aplicações que buscam dados para serem processados ​​como parte da inferência de modelos, como no caso da Geração Aumentada de Recuperação (RAG).

O exemplo apresenta um mecanismo de busca em um documento PDF. Isso nos permitirá recuperar trechos no PDF que sejam semelhantes a uma consulta de entrada. O guia também inclui uma implementação mínima de RAG sobre o mecanismo de busca.

Este exemplo requer os pacotes `langchain-community` e `pypdf`:

In [ ]:
!pip install -qU langchain-community pypdf

**Carregando documentos**


Vamos carregar um PDF em uma sequência de objetos `Document`. Aqui está um PDF de exemplo — [um formulário 10-K da Nike de 2023](https://github.com/langchain-ai/langchain/raw/v0.3/docs/docs/example_data/nke-10k-2023.pdf). Podemos consultar a documentação do LangChain para obter informações sobre os carregadores de documentos PDF disponíveis.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/content/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


`PyPDFLoader` carrega um objeto `Document` por página PDF. Para cada um, podemos acessar facilmente:

* O conteúdo da página em formato de string;

* Os metadados contendo o nome do arquivo e o número da página.

In [ ]:
print(f"{docs[0].page_content[:100]}\n")

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K




In [ ]:
print(docs[0].metadata)

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '/content/nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


***`Splitting`***

Para fins de recuperação de informações e de perguntas e respostas subsequentes, uma página pode ser uma representação muito grosseira. Nosso objetivo final será recuperar objetos `Document` que respondam a uma consulta de entrada, e dividir ainda mais nosso PDF ajudará a garantir que os significados das partes relevantes do documento não sejam "distorcidos" pelo texto circundante.

Podemos usar divisores de texto (*`text splitters`*) para esse propósito. Aqui, usaremos um divisor de texto simples que particiona com base em caracteres. Dividiremos nossos documentos em blocos de 1000 caracteres com 200 caracteres de sobreposição entre os blocos. A sobreposição ajuda a mitigar a possibilidade de separar uma declaração de um contexto importante relacionado a ela. Usamos o `RecursiveCharacterTextSplitter`, que dividirá recursivamente o documento usando separadores comuns, como novas linhas, até que cada bloco tenha o tamanho apropriado. Este é o divisor de texto recomendado para casos de uso de texto genérico.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

516


***`Embeddings`***


A busca vetorial (*`vector search`*) é uma maneira comum de armazenar e pesquisar dados não estruturados (como texto não estruturado). A ideia é armazenar vetores numéricos associados ao texto. Dada uma consulta, podemos incorporá-la como um vetor da mesma dimensão e usar métricas de similaridade vetorial (como similaridade de cosseno) para identificar textos relacionados.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=OPENAI_API_KEY)

In [ ]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[0.009298405610024929, -0.01608305238187313, 0.00028412663959898055, 0.0064094592817127705, 0.020547788590192795, -0.03926966339349747, -0.007359934970736504, 0.04102053865790367, -0.008072791621088982, 0.05998003110289574]


Munidos de um modelo para gerar representações vetoriais de texto, podemos então armazená-las em uma estrutura de dados especial que suporta buscas de similaridade eficientes.

***`Vector stores`***

Os objetos `VectorStore` do **LangChain** contêm métodos para adicionar objetos de texto e `Document` ao armazenamento e para consultá-los usando várias métricas de similaridade. Eles são frequentemente inicializados com modelos de incorporação, que determinam como os dados de texto são traduzidos em vetores numéricos.

In [ ]:
!pip install -qU langchain-core

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

Após instanciarmos nosso armazenamento de vetores, podemos agora indexar os documentos.

In [ ]:
ids = vector_store.add_documents(documents=all_splits)

Cada documento inserido recebe um ID único, que é retornado na variável `ids`.

In [ ]:
print(ids[:3])

['4ee05d6e-3028-4245-99f6-a5a76d3607ce', 'b51ce74f-fe40-4572-ba20-aa3653c6cbc4', 'e4c70d67-6b69-46ee-a68d-634470e07840']


**Uso**

Os *`embeddings`* geralmente representam o texto como um vetor "denso", de forma que textos com significados semelhantes fiquem geometricamente próximos. Isso nos permite recuperar informações relevantes apenas inserindo uma pergunta, sem conhecimento de quaisquer palavras-chave específicas usadas no documento. Retorna documentos com base na similaridade a uma consulta em formato de string:

In [ ]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?",
    k=1,
)

print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 

Esse método faz uma busca semântica dentro do banco vetorial.

* Ele pega a consulta ("*How many distribution centers does Nike have in the US?*"),

* Gera o *`embedding`* (vetor numérico) dessa pergunta,

* E compara esse vetor com todos os *`embeddings`* armazenados no `vector_store` (que você adicionou antes com `add_documents`).

Ele calcula a similaridade vetorial (normalmente usando similaridade por cosseno), e retorna os documentos mais parecidos em significado.

Parâmetro `k=1`

Esse argumento define quantos resultados você quer retornar.

`k=1` → retorna apenas o documento mais similar

In [ ]:
from IPython.display import display, Markdown

display(Markdown(results[0].page_content))

operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,

In [ ]:
print(results[0].metadata)

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': '/content/nke-10k-2023.pdf', 'total_pages': 107, 'page': 26, 'page_label': '27', 'start_index': 804}


Consulta assíncrona:

In [ ]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this "Annual Report"), the terms "we," "us," "our,"
"NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, equipment, accessories and services. NIKE is
the largest seller of athletic footwear and apparel in the world. We sell our products through NIKE Direct operations, which are comprised of both NIKE-owned retail stores
and sales through our digital platforms (also referred to as "NIKE Brand Digital"), to retail accounts and to a mix of independent distributors, licensees and sales' metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'cr

Retorna *`scores`*:

In [ ]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.6887236001856845

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTSThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' metad

Retornar documentos com base na similaridade a uma consulta:

In [ ]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:'

**Referência**

> https://docs.langchain.com/oss/python/langchain/knowledge-base
